In [1]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 19.9 MB/s eta 0:00:00


In [2]:
import os
import rasterio
import pandas as pd
import numpy as np
import glob

In [2]:
# Create a folder called 'downloads' in the current directory
os.makedirs('downloads', exist_ok=True)

In [5]:
data = pd.read_excel('CH4.xlsx')
data.head()

,Link
0,https://dljsq618eotzp.cloudfront.net/tm54dvar-...
1,https://dljsq618eotzp.cloudfront.net/tm54dvar-...
2,https://dljsq618eotzp.cloudfront.net/tm54dvar-...
3,https://dljsq618eotzp.cloudfront.net/tm54dvar-...
4,https://dljsq618eotzp.cloudfront.net/tm54dvar-...


In [8]:
file_links = data['Link'].to_list()
# Directory to save files
output_dir = 'downloads'

# Loop through all the links and download each file
for link in file_links:
    os.system(f"wget -P {output_dir} {link}")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Copy the 'downloads' folder to a specific location in your Google Drive
!cp -r downloads /content/drive/MyDrive/

In [4]:
def tif_to_dataframe(tif_file_path):
    with rasterio.open(tif_file_path) as src:
        # Read the data as a numpy array
        data = src.read(1)  # Read the first band, adjust if needed for multiple bands

        # Get the transform (mapping between array indices and coordinates)
        transform = src.transform

        # Create empty lists to store the x, y, and value data
        x_coords = []
        y_coords = []
        values = []

        # Loop through each row and column in the numpy array
        for row in range(data.shape[0]):
            for col in range(data.shape[1]):
                # Get the value at the current pixel
                value = data[row, col]

                # Get the x, y coordinates of the current pixel
                x, y = transform * (col, row)

                # Append the coordinates and value to the respective lists
                x_coords.append(x)
                y_coords.append(y)
                values.append(value)

        # Create a DataFrame from the coordinates and values
        df = pd.DataFrame({
            'x': x_coords,
            'y': y_coords,
            'value': values
        })

        return df


In [6]:
tif_files[:10]

['/content/drive/MyDrive/downloads/methane_emis_fossil_200509.tif',
 '/content/drive/MyDrive/downloads/methane_emis_fossil_200510.tif',
 '/content/drive/MyDrive/downloads/methane_emis_fossil_200511.tif',
 '/content/drive/MyDrive/downloads/methane_emis_fossil_200512.tif',
 '/content/drive/MyDrive/downloads/methane_emis_fossil_200601.tif',
 '/content/drive/MyDrive/downloads/methane_emis_fossil_200602.tif',
 '/content/drive/MyDrive/downloads/methane_emis_fossil_200603.tif',
 '/content/drive/MyDrive/downloads/methane_emis_fossil_200604.tif',
 '/content/drive/MyDrive/downloads/methane_emis_fossil_200605.tif',
 '/content/drive/MyDrive/downloads/methane_emis_fossil_200606.tif']

In [10]:
'/content/drive/MyDrive/downloads/methane_emis_fossil_200602.tif'.split('/')[-1].split('_')[-1].replace('.tif', '')[:4]

'2006'

In [12]:
def merge_df(df1, df2, band_name):
    return pd.merge(df1, df2.rename(columns={'value': band_name}), on=['x', 'y'])

# List of .tif files
tif_files = glob.glob('/content/drive/MyDrive/downloads/*')

# Initialize a dataframe to store the merged result
df = None

# Process each file
for i, file_path in enumerate(tif_files):
    # Use the tif_to_dataframe function to read the .tif file
    temp_df = tif_to_dataframe(file_path)

    # Name each band based on the file index (e.g., band_1, band_2, etc.)
    band_name = file_path.split('/')[-1].split('_')[-1].replace('.tif', '')
    year, month = band_name[:4], band_name[4:]
    combo = f'{year}_{month}'
    band_name = f'band_{i+1}'

    # Merge the dataframes based on 'x' and 'y'
    if df is None:
        df = temp_df.rename(columns={'value': combo})  # First dataframe sets the base
    else:
        df = merge_df(df, temp_df, combo)


    if i>=24:
      break

df

,x,y,2005_09,2005_10,2005_11,2005_12,2006_01,2006_02,2006_03,2006_04,...,2006_12,2007_01,2007_02,2007_03,2007_04,2007_05,2007_06,2007_07,2007_08,2007_09
0,-180.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-179.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-178.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-177.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-176.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64795,175.0,-89.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64796,176.0,-89.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64797,177.0,-89.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64798,178.0,-89.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df.melt(id_vars=['x', 'y'], var_name='date', value_name='ch4_emission')

,x,y,date,ch4_emission
0,-180.0,90.0,2005_09,0.0
1,-179.0,90.0,2005_09,0.0
2,-178.0,90.0,2005_09,0.0
3,-177.0,90.0,2005_09,0.0
4,-176.0,90.0,2005_09,0.0
...,...,...,...,...
1619995,175.0,-89.0,2007_09,0.0
1619996,176.0,-89.0,2007_09,0.0
1619997,177.0,-89.0,2007_09,0.0
1619998,178.0,-89.0,2007_09,0.0


In [1]:
!wget https://dljsq618eotzp.cloudfront.net/oco2-mip-co2budget-yeargrid-v1/pilot_topdown_Crop_CO2_Budget_grid_v1_2015.tif
!wget https://dljsq618eotzp.cloudfront.net/oco2-mip-co2budget-yeargrid-v1/pilot_topdown_Crop_CO2_Budget_grid_v1_2016.tif
!wget https://dljsq618eotzp.cloudfront.net/oco2-mip-co2budget-yeargrid-v1/pilot_topdown_Crop_CO2_Budget_grid_v1_2017.tif
!wget https://dljsq618eotzp.cloudfront.net/oco2-mip-co2budget-yeargrid-v1/pilot_topdown_Crop_CO2_Budget_grid_v1_2018.tif
!wget https://dljsq618eotzp.cloudfront.net/oco2-mip-co2budget-yeargrid-v1/pilot_topdown_Crop_CO2_Budget_grid_v1_2019.tif
!wget https://dljsq618eotzp.cloudfront.net/oco2-mip-co2budget-yeargrid-v1/pilot_topdown_Crop_CO2_Budget_grid_v1_2020.tif

--2024-10-04 21:44:49--  https://dljsq618eotzp.cloudfront.net/oco2-mip-co2budget-yeargrid-v1/pilot_topdown_Crop_CO2_Budget_grid_v1_2015.tif
Resolving dljsq618eotzp.cloudfront.net (dljsq618eotzp.cloudfront.net)... 108.139.38.91, 108.139.38.189, 108.139.38.161, ...
Connecting to dljsq618eotzp.cloudfront.net (dljsq618eotzp.cloudfront.net)|108.139.38.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519324 (507K) [binary/octet-stream]
Saving to: ‘pilot_topdown_Crop_CO2_Budget_grid_v1_2015.tif’

pilot_topdown_Crop_ 100%[===================>] 507.15K  2.07MB/s    in 0.2s    

2024-10-04 21:44:50 (2.07 MB/s) - ‘pilot_topdown_Crop_CO2_Budget_grid_v1_2015.tif’ saved [519324/519324]

--2024-10-04 21:44:50--  https://dljsq618eotzp.cloudfront.net/oco2-mip-co2budget-yeargrid-v1/pilot_topdown_Crop_CO2_Budget_grid_v1_2016.tif
Resolving dljsq618eotzp.cloudfront.net (dljsq618eotzp.cloudfront.net)... 108.139.38.91, 108.139.38.189, 108.139.38.161, ...
Connecting to dljsq618eotz

In [6]:
tif_to_dataframe('/content/pilot_topdown_Crop_CO2_Budget_grid_v1_2018.tif')

,x,y,value
0,-180.0,90.0,-9999.0
1,-179.0,90.0,-9999.0
2,-178.0,90.0,-9999.0
3,-177.0,90.0,-9999.0
4,-176.0,90.0,-9999.0
...,...,...,...
64795,175.0,-89.0,-9999.0
64796,176.0,-89.0,-9999.0
64797,177.0,-89.0,-9999.0
64798,178.0,-89.0,-9999.0


In [11]:
def merge_df(df1, df2, band_name):
    return pd.merge(df1, df2.rename(columns={'value': band_name}), on=['x', 'y'])

# List of .tif files
tif_files = [
    'pilot_topdown_Crop_CO2_Budget_grid_v1_2015.tif',
    'pilot_topdown_Crop_CO2_Budget_grid_v1_2016.tif',
    'pilot_topdown_Crop_CO2_Budget_grid_v1_2017.tif',
    'pilot_topdown_Crop_CO2_Budget_grid_v1_2018.tif',
    'pilot_topdown_Crop_CO2_Budget_grid_v1_2019.tif',
]

# Directory where the files are stored
directory = '/content'

# Initialize a dataframe to store the merged result
df = None

# Process each file
for i, file_name in enumerate(tif_files):
    file_path = os.path.join(directory, file_name)

    # Use the tif_to_dataframe function to read the .tif file
    temp_df = tif_to_dataframe(file_path)

    # Name each band based on the file index (e.g., band_1, band_2, etc.)
    band_name = f'band_{i+1}'

    # Merge the dataframes based on 'x' and 'y'
    if df is None:
        df = temp_df.rename(columns={'value': band_name})  # First dataframe sets the base
    else:
        df = merge_df(df, temp_df, band_name)

df

,x,y,band_1,band_2,band_3,band_4,band_5
0,-180.0,90.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,-179.0,90.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,-178.0,90.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,-177.0,90.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,-176.0,90.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...
64795,175.0,-89.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
64796,176.0,-89.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
64797,177.0,-89.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
64798,178.0,-89.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [12]:
df.sample(10)

,x,y,band_1,band_2,band_3,band_4,band_5
59862,-78.0,-76.0,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
35412,-48.0,-8.0,2.309417,2.086628,2.400156,2.277610,2.166812
41925,-15.0,-26.0,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
34566,-174.0,-6.0,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
57789,9.0,-70.0,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
46157,-103.0,-38.0,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
47824,124.0,-42.0,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
5138,-82.0,76.0,0.146995,0.163774,0.145107,0.143397,0.142253
38665,-35.0,-17.0,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
4540,40.0,78.0,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
